# Test Upload Graph

This notebook tests the Upload graph by connecting to the LangGraph server.

**Prerequisites:**
1. Start the LangGraph dev server: `cd backend && npm run dev`
2. Make sure the server is running at `http://localhost:2024`

In [1]:
# Install required packages (run once)
# %pip install langgraph-sdk
# Ollama is running locally for this test.

In [2]:
from langgraph_sdk import get_client

# Connect to the local LangGraph server
# Make sure you've started it with: cd backend && npm run dev
client = get_client(url="http://localhost:2024")

print("✅ Connected to LangGraph server")


✅ Connected to LangGraph server


In [3]:
# Create a new thread (conversation session)
thread = await client.threads.create()
print(f"✅ Created thread: {thread['thread_id']}")


✅ Created thread: 51d6dfeb-771f-46e6-8313-814a0dcac86b


In [4]:
from langchain_community.document_loaders import PyPDFLoader

# Initialize allDocs list if not already defined
if 'allDocs' not in globals():
    allDocs = []

# Load PDF using LangChain's PyPDFLoader
loader = PyPDFLoader('Sample-Accounting-Income-Statement.pdf')
docs = loader.load()

# Add documents to allDocs list
allDocs.extend(docs)
print(f"✅ Loaded {len(docs)} documents. Total documents: {len(allDocs)}")

✅ Loaded 7 documents. Total documents: 7


In [92]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

splitter = RecursiveCharacterTextSplitter(
    chunk_size=30,
    chunk_overlap=10,
)


doc = '''The Fellowship moved quietly through the dark forest, their footsteps muffled by the thick layer of fallen leaves.
Aragorn led the way, his keen eyes scanning the shadows for any sign of danger ahead.
Behind him, Frodo clutched the Ring, feeling its weight grow heavier with each step toward Mordor.'''

# Convert string to Document object before splitting
doc_obj = Document(page_content=doc)
splitDocs = splitter.split_documents([doc_obj])

len(splitDocs)
splitDocs

[Document(metadata={}, page_content='The Fellowship moved quietly'),
 Document(metadata={}, page_content='quietly through the dark'),
 Document(metadata={}, page_content='the dark forest, their'),
 Document(metadata={}, page_content='their footsteps muffled by'),
 Document(metadata={}, page_content='by the thick layer of fallen'),
 Document(metadata={}, page_content='of fallen leaves.'),
 Document(metadata={}, page_content='Aragorn led the way, his keen'),
 Document(metadata={}, page_content='his keen eyes scanning the'),
 Document(metadata={}, page_content='the shadows for any sign of'),
 Document(metadata={}, page_content='sign of danger ahead.'),
 Document(metadata={}, page_content='Behind him, Frodo clutched'),
 Document(metadata={}, page_content='clutched the Ring, feeling'),
 Document(metadata={}, page_content='feeling its weight grow'),
 Document(metadata={}, page_content='grow heavier with each step'),
 Document(metadata={}, page_content='each step toward Mordor.')]

In [95]:
result = await client.runs.create(
      thread_id=thread['thread_id'],
      assistant_id="upload_graph",
      input={
        "docs": allDocs[:1],
      },
      config={
        "configurable": {
          "retrieverProvider": "supabase",
          "filterKwargs": {},
          "k": 5,
        },
      },
    )

In [ ]:
# Cancel the run
result = await client.runs.get(
    thread_id=result.get('thread_id'),
    run_id=result.get('run_id')
)
await client.runs.cancel(thread_id=result.get('thread_id'), run_id=result.get('run_id'))

In [ ]:
# Get the run status
result = await client.runs.get(
    thread_id=result.get('thread_id'),
    run_id=result.get('run_id')
)
result.get('status')

'success'